In [1]:
!pip install pyomo -q
!pip install mistralai -q
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00


In [2]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [3]:
API_KEY = ''

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 2
MODEL_TEMPERATURE = 0.7

SYSTEM_PROMPT_1 = """Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."""
SYSTEM_PROMPT_2 = """Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."""
PROBLEM_DESCRIPTION = """You are a chemist in charge of a process that requires two liquids to reach the end product. Your goal is to maximize the quality of the product in the end, which is measured by the difference between the Napierian logarithm of the product of the litres used of each component and the cube root of the summation of the first component litres as well as double the second component litres. It is required that the squared difference between the litres employed of each component and the mean litres utilised of both components is below ten. This is necessary for the obtained set to be stable. Furthermore, the mixture tank available to you can only fit 80 l. Please formulate a mathematical optimization model for this problem."""


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-06-14 13:54:48.124972


## Step 1 - Generate Mathematical Formulation

In [4]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [5]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_1_text = response_1.choices[0].message.content

In [6]:
Markdown(response_1_text)

To formulate a mathematical optimization model for this problem, we first define the parameters, decision variables, objective function, and constraints.

Parameters:
- Mean liters utilized of both components = M

Decision variables:
- Liters of first component = x₁
- Liters of second component = x₂

Objective function:
The objective is to maximize the quality of the product, which is measured by the difference between the Napierian logarithm of the product of the liters used of each component and the cube root of the summation of the first component liters as well as double the second component liters. Therefore, the objective function is:

Maximize Z = ln(x₁ * x₂) - (x₁ + 2x₂)^(1/3)

Constraints:
1. The squared difference between the liters employed of each component and the mean liters utilized of both components is below ten. This ensures that the obtained set is stable.

   Constraint: (x₁ - M)^2 + (x₂ - M)^2 ≤ 10

2. The mixture tank available can only fit 80L.

   Constraint: x₁ + x₂ ≤ 80

3. Non-negativity constraints: The liters of each component used must be non-negative.

   Constraints: x₁ ≥ 0, x₂ ≥ 0

Combining all these elements, the optimization model is as follows:

Maximize Z = ln(x₁ * x₂) - (x₁ + 2x₂)^(1/3)
Subject to:
(x₁ - M)^2 + (x₂ - M)^2 ≤ 10
x₁ + x₂ ≤ 80
x₁ ≥ 0
x₂ ≥ 0

In [7]:
print(response_1_text)

To formulate a mathematical optimization model for this problem, we first define the parameters, decision variables, objective function, and constraints.

Parameters:
- Mean liters utilized of both components = M

Decision variables:
- Liters of first component = x₁
- Liters of second component = x₂

Objective function:
The objective is to maximize the quality of the product, which is measured by the difference between the Napierian logarithm of the product of the liters used of each component and the cube root of the summation of the first component liters as well as double the second component liters. Therefore, the objective function is:

Maximize Z = ln(x₁ * x₂) - (x₁ + 2x₂)^(1/3)

Constraints:
1. The squared difference between the liters employed of each component and the mean liters utilized of both components is below ten. This ensures that the obtained set is stable.

   Constraint: (x₁ - M)^2 + (x₂ - M)^2 ≤ 10

2. The mixture tank available can only fit 80L.

   Constraint: x₁

## Step 2 - Generate the Pyomo Code

In [8]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [9]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [10]:
Markdown(response_2_text)

Here is a Python code using Pyomo to solve the optimization problem:

```python
# Import the necessary packages
from pyomo.environ import *
import math

# Initialize the model
model = ConcreteModel()

# Parameters
model.M = 30  # Sample data for mean liters utilized of both components

# Decision variables
model.x1 = Var(within=NonNegativeReals)
model.x2 = Var(within=NonNegativeReals)

# Objective function
def obj_rule(model):
    return log(model.x1 * model.x2) - pow(model.x1 + 2*model.x2, 1/3)
model.obj = Objective(rule=obj_rule, sense=maximize)

# Constraints
def stable_set_rule(model):
    return pow(model.x1 - model.M, 2) + pow(model.x2 - model.M, 2) <= 10
model.stable_set = Constraint(rule=stable_set_rule)

def tank_capacity_rule(model):
    return model.x1 + model.x2 <= 80
model.tank_capacity = Constraint(rule=tank_capacity_rule)

# Solve the model
solver = SolverFactory('ipopt')
results = solver.solve(model)

# Print the results
print("Objective value:", model.obj())
print("x1 =", model.x1())
print("x2 =", model.x2())
```

Please note that the solver used here is 'ipopt'. Ensure that you have the 'ipopt' solver installed and available in your environment to run this code.

The parameter M is set to 30 as sample data. You can replace it with your actual mean liters utilized for both components.

In [11]:
print(response_2_text)

Here is a Python code using Pyomo to solve the optimization problem:

```python
# Import the necessary packages
from pyomo.environ import *
import math

# Initialize the model
model = ConcreteModel()

# Parameters
model.M = 30  # Sample data for mean liters utilized of both components

# Decision variables
model.x1 = Var(within=NonNegativeReals)
model.x2 = Var(within=NonNegativeReals)

# Objective function
def obj_rule(model):
    return log(model.x1 * model.x2) - pow(model.x1 + 2*model.x2, 1/3)
model.obj = Objective(rule=obj_rule, sense=maximize)

# Constraints
def stable_set_rule(model):
    return pow(model.x1 - model.M, 2) + pow(model.x2 - model.M, 2) <= 10
model.stable_set = Constraint(rule=stable_set_rule)

def tank_capacity_rule(model):
    return model.x1 + model.x2 <= 80
model.tank_capacity = Constraint(rule=tank_capacity_rule)

# Solve the model
solver = SolverFactory('ipopt')
results = solver.solve(model)

# Print the results
print("Objective value:", model.obj())
print("x1 

### Code Executability

In [12]:
from pyomo.environ import *
import math

# Initialize the model
model = ConcreteModel()

# Parameters
model.M = 30  # Sample data for mean liters utilized of both components

# Decision variables
model.x1 = Var(within=NonNegativeReals)
model.x2 = Var(within=NonNegativeReals)

# Objective function
def obj_rule(model):
    return log(model.x1 * model.x2) - pow(model.x1 + 2*model.x2, 1/3)
model.obj = Objective(rule=obj_rule, sense=maximize)

# Constraints
def stable_set_rule(model):
    return pow(model.x1 - model.M, 2) + pow(model.x2 - model.M, 2) <= 10
model.stable_set = Constraint(rule=stable_set_rule)

def tank_capacity_rule(model):
    return model.x1 + model.x2 <= 80
model.tank_capacity = Constraint(rule=tank_capacity_rule)

# Solve the model
solver = SolverFactory('ipopt')
results = solver.solve(model)

# Print the results
print("Objective value:", model.obj())
print("x1 =", model.x1())
print("x2 =", model.x2())

Objective value: 2.3694005522892816
x1 = 33.157376883507375
x2 = 30.17598341986668


### Solution Correctness

In [11]:
from pyomo.environ import *
import math

# Initialize the model
model = ConcreteModel()

# Parameters
model.M = 30  # Sample data for mean liters utilized of both components

# Decision variables
model.x1 = Var(within=NonNegativeReals)
model.x2 = Var(within=NonNegativeReals)

# Objective function
def obj_rule(model):
    return log(model.x1 * model.x2) - pow(model.x1 + 2*model.x2, 1/3)
model.obj = Objective(rule=obj_rule, sense=maximize)

# Constraints
def stable_set_rule(model):
    return pow(model.x1 - model.M, 2) + pow(model.x2 - model.M, 2) <= 10
model.stable_set = Constraint(rule=stable_set_rule)

def tank_capacity_rule(model):
    return model.x1 + model.x2 <= 80
model.tank_capacity = Constraint(rule=tank_capacity_rule)

# Solve the model
solver = SolverFactory('ipopt')
results = solver.solve(model)

# Print the results
print("Objective value:", model.obj())
print("x1 =", model.x1())
print("x2 =", model.x2())